In [1]:
include("src\\building_tree.jl")
include("src\\utilities.jl")
include("src\\merge.jl")
using Debugger
using DataFrames, Random
using DataStructures
using StatsBase
using Statistics
using LinearAlgebra
using LazySets
using Polyhedra
using InvertedIndices
# using Plots

In [2]:
dataSetName = "iris"

"iris"

In [3]:
include("./data/" * dataSetName * ".txt")
# Ramener chaque caractéristique sur [0, 1]
reducedX = Matrix{Float64}(X)
for j in 1:size(X, 2)
    reducedX[:, j] .-= minimum(X[:, j])
    reducedX[:, j] ./= maximum(X[:, j])
end

train, test = train_test_indexes(length(Y))
X_train = reducedX[train,:]
Y_train = Y[train]
X_test = reducedX[test,:]
Y_test = Y[test]
classes = unique(Y)

println(" (train size ", size(X_train, 1), ", test size ", size(X_test, 1), ", ", size(X_train, 2), ", features count: ", size(X_train, 2), ")")

# Temps limite de la

 (train size 120, test size 30, 4, features count: 4)


In [4]:
time_limit = 10
D = 3
x = X_train
y = Y_train

120-element Vector{Int64}:
 3
 1
 3
 1
 1
 1
 1
 1
 2
 1
 ⋮
 3
 1
 2
 3
 1
 1
 1
 2
 1

In [5]:
n = length(Y_train)
m = length(x[1,:])
clusters = Vector{Cluster}([])
for dataId in 1:size(x,1)
    push!(clusters, Cluster(dataId, x,y))
end
clusterId = collect(1:n) #On obtient un vecteur 1,2..., qui correspond pour chaque cluster à son clusterId
distances = Vector{Distance}([])
for id1 in 1:n-1
    for id2 in id1+1:n
        push!(distances, Distance(id1, id2, x))
    end
end
sort!(distances, by = v ->v.distance)
remainingClusters=n
distanceId = 1
n_epochs = 1
c1_bis = Nothing
c2_bis = Nothing
i = 1
a=i
max_elements_small_classes = 5
num_clusters = 10

while remainingClusters>= num_clusters
    distance = distances[distanceId]
    cId1 =clusterId[distance.ids[1]]
    cId2 = clusterId[distance.ids[2]]
    if cId1 != cId2
        c1 = clusters[cId1]
        c2 = clusters[cId2]
        count_classes_in_clusters = StatsBase.countmap([y[c2.dataIds]; y[c1.dataIds]])
        df = DataFrame()
        df."classes" = collect(keys(count_classes_in_clusters))
        df."count_classes" = collect(values(count_classes_in_clusters))
        df = sort(df, [:count_classes], rev=true)
        # if sum(df[df."count_classes".!=maximum(df."count_classes"),:]."count_classes") <= max_elements_small_classes
        if sum(df[df."classes".!=df[1,1],"count_classes"])<= max_elements_small_classes
            remainingClusters -=1
            if remainingClusters < num_clusters
                break
            end
            merge!(c1, c2) #On merge les 2 clusters
            for id in c2.dataIds 
                clusterId[id]= cId1 #On modifie le clusterId dans la serie pour le cluster_2, on lui affecte le cluster_1
            end
            # Vider le second cluster
            empty!(clusters[cId2].dataIds)
        end

    end
    distanceId += 1
    # if i % 50 == 0
    # #     println(remainingClusters)
    # #     # print("\n")
    # # end
end

In [6]:
df_clusters = DataFrame()
df_clusters."cluster_id" = collect(keys(StatsBase.countmap(clusterId)))
df_clusters."number_elements" = collect(values(StatsBase.countmap(clusterId)))
higher_than_threshold(value::Int64) = value >= 10
clusters_to_treat = filter(:"number_elements"=> higher_than_threshold, df_clusters)."cluster_id"


3-element Vector{Int64}:
 41
 38
  9

In [7]:
mean(y[clusters[38].dataIds])

1.0

Reprendre scaffold dans slides cooperative optimization : regarder le papier, on a un petit problème au niveau des résultats et de la mise à jour des paramètres

In [38]:
cluster_interest = clusters_to_treat[3]
count_classes_in_clusters = StatsBase.countmap(y[findall(x->x==cluster_interest, clusterId)])
y_cluster = y[findall(x->x==cluster_interest, clusterId)]
df = DataFrame()
df."classes" = collect(keys(count_classes_in_clusters))
df."count_classes" = collect(values(count_classes_in_clusters))
df = sort(df, [:count_classes], rev=true)
main_class = 0
if size(df)[1]>1
    main_class = df[1,1]
    print("several classes in cluster")
else 
    main_class = df[1,1]
    print("only class in cluster")
end   
points_in_cluster = findall(x->x==cluster_interest, clusterId)
points_reckon_with = Vector{Int64}([])
points_sideline = Vector{Int64}([])
for (i, point) in enumerate(points_in_cluster)
    if y[point] != main_class
        push!(points_sideline,i )
    else
        push!(points_reckon_with, i)
    end
end
matrix_interest = clusters[cluster_interest].x
size(matrix_interest, 1)
vectors_cluster = [matrix_interest[i,:] for i in 1:size(matrix_interest,1)]
vectors_cluster[points_reckon_with]
hull = LazySets.convex_hull(vectors_cluster[points_reckon_with])
for i in 1:length(points_sideline)
    println(i, " ",element(Singleton(vectors_cluster[points_sideline[i]])) ∈ VPolytope(hull))
    if element(Singleton(vectors_cluster[points_sideline[i]])) ∈ VPolytope(hull)
        push!(points_reckon_with,  points_sideline[i])
    end
end
sort!(points_reckon_with)
points_to_be_taken_out_cluster = points_sideline[points_sideline .∉ Ref(points_reckon_with)]

new_clusters = Vector{Cluster}([])
point_cluster  = points_reckon_with[1]
trimmed_cluster = Cluster(point_cluster, matrix_interest, y_cluster)
for point_cluster in points_reckon_with[2:end]
    new_cluster = Cluster(point_cluster, matrix_interest, y_cluster)
    merge!(trimmed_cluster, new_cluster)
end

for point_cluster in points_to_be_taken_out_cluster
    new_cluster = Cluster(point_cluster, matrix_interest, y_cluster)
    push!(clusters, new_cluster)
end
clusters[cluster_interest] = trimmed_cluster



several classes in clusterglp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution

1 true
2 true
3 false
4 false
5 false


Cluster([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  28, 29, 31, 32, 33, 34, 35, 37, 39, 40], [0.19444444444444448, 0.0, 0.3389830508474576, 0.375], [0.7222222222222222, 0.5, 0.7796610169491525, 0.7916666666666666], [0.5555555555555555 0.20833333333333331 0.6610169491525424 0.5833333333333334; 0.4999999999999999 0.25 0.7796610169491525 0.5416666666666666; … ; 0.19444444444444448 0.12499999999999992 0.38983050847457623 0.375; 0.22222222222222213 0.20833333333333331 0.3389830508474576 0.4166666666666667], 2)

In [14]:
distance = distances[distanceId]
cId1 =clusterId[distance.ids[1]]
cId2 = clusterId[distance.ids[2]]
c1 = clusters[cId1]
c2 = clusters[cId2]
count_classes_in_clusters = StatsBase.countmap([y[c2.dataIds]; y[c1.dataIds]])
df = DataFrame()
df."classes" = collect(keys(count_classes_in_clusters))
df."count_classes" = collect(values(count_classes_in_clusters))
df = sort(df, [:count_classes], rev=true)

Row,classes,count_classes
,Int64,Int64
1,3,22


In [15]:
using QHull
using LazySets
using Polyhedra
x_test = rand(200, 2)
ch = chull(x_test)
ch.points         # original points
ch.vertices       # indices to line segments forming the convex hull
ch.simplices      # the simplexes forming the convex hull
show(ch)

# Constructs a V-representation of 10 random points in 2 dimension

v = vrep(x_test)

# Constructs a polyhedon from this V-representation with the QHull library
p = polyhedron(v, QHull.Library())

Chull{Float64}

([0.9701902381663272 0.7669756716870548; 0.1924316131638032 0.13804553291012567; 0.03440015986770406 0.6243709879936329; 0.6767039719919596 0.5178321731913068; 0.11299318504818134 0.3996668264890465; 0.16489890340914626 0.8323944665177172; 0.8069622969868445 0.6044024321042605; 0.03057039394926897 0.19819610321586212; 0.6689163952554441 0.020304794840812934; 0.2852518757825253 0.7992032247538037; 0.8630983402270727 0.009163791471611171; 0.7426243199962828 0.37792793187827944; 0.1258296509779252 0.6392434400411919; 0.3563461608705246 0.5054604292112024; 0.4106106927251686 0.10216107335473268; 0.46406965266479616 0.1589518880659636; 0.26187938116585874 0.17800126806362837; 0.36986660400669347 0.9618048068583325; 0.5461963986546966 0.6425461373701451; 0.2661180892659998 0.213786475619685; 0.33910020146636555 0.2672475052639842; 0.3263605545945688 0.012941965671099731; 0.528573979061918 0.7360595443555475; 0.4266302244339928 0.3909564688545426; 0.4710443305520664 0.3322019409658922; 0.3074

 0.8043619817035292; 0.06436672156832768 0.381514968889046; 0.9103149566005841 0.3566771401726764; 0.10141248739594 0.29275560386916955; 0.17906332012630388 0.9398893090886787; 0.8876593621726635 0.3216787268615726; 0.9335443164730844 0.4975029537723301; 0.3781613780416063 0.09847372201739624; 0.6332541164933523 0.9482477707426582; 0.09487272844041683 0.9104966196887814; 0.03865108343161128 0.5028642978358073; 0.9518872306786422 0.7592013670590184; 0.4354987833150976 0.44929390508448985; 0.02050974802390282 0.5569125466735454; 0.9385067282348369 0.6807542812441446; 0.18118749311600924 0.5380660568067087; 0.8183391386925531 0.9888148935893187; 0.8309408609337939 0.5314645877351309; 0.8574507218400615 0.7363678713205201; 0.6265192622621533 0.4568206227063638; 0.9353287044147149 0.5241778138858297; 0.35310740145562436 0.1437536496201811; 0.4279365788621564 0.9559808744254069; 0.8512964901092949 0.4479906157948381; 0.7085135448203553 0.8550190350694076; 0.08549775163945206 0.19038674042164

[121, 129, 111, 8, 42, 31, 11, 130, 45, 55, 1, 70, 155, 120], 

[[121, 129], [11, 31], [120, 155], [120, 121], [55, 45], [55, 1], [70, 155], [70, 1], [111, 129], [111, 8], [42, 31], [42, 8], [130, 45], [130, 11]], [-0.995828589717026 0.09124373897533537 -0.045223337243135854; -0.0018411184999350973 -0.9999983051398983 0.010752842261721966; -0.00013792347844605805 0.999999990488557 -0.9887020160037052; -0.0005303870298955773 0.9999998593447894 -0.988555414060862; 0.9997259812840736 0.02340859554941462 -0.990282791488555; 0.9994049446775759 0.03449284786751255 -0.9960680964629273; 0.12457380701913431 0.9922103439315474 -1.0830559875925851; 0.9797134819226163 0.20040332666665847 -1.1042129324397454; -0.9972038680226734 -0.07472914826637907 0.03892112773894751; -0.9893656300414866 -0.14544981984386254 0.05907288457673767; -0.6204740510472468 -0.7842269773331052 0.07475530940515307; -0.9718286873657113 -0.23568835867101806 0.07642170008590897; 0.9917537265475882 -0.12815828447264563 -0.9629409815884865; 0.5221849195296828 -0.8528322870387699 -0.44288176

Polyhedron QHull.Polyhedron:
200-element iterator of Vector{Float64}:
 [0.9701902381663272, 0.7669756716870548]
 [0.1924316131638032, 0.13804553291012567]
 [0.03440015986770406, 0.6243709879936329]
 [0.6767039719919596, 0.5178321731913068]
 [0.11299318504818134, 0.3996668264890465]
 [0.16489890340914626, 0.8323944665177172]
 [0.8069622969868445, 0.6044024321042605]
 [0.03057039394926897, 0.19819610321586212]
 [0.6689163952554441, 0.020304794840812934]
 [0.2852518757825253, 0.7992032247538037]
 [0.8630983402270727, 0.009163791471611171]
 [0.7426243199962828, 0.37792793187827944]
 [0.1258296509779252, 0.6392434400411919]
 [0.3563461608705246, 0.5054604292112024]
 [0.4106106927251686, 0.10216107335473268]
 [0.46406965266479616, 0.1589518880659636]
 [0.26187938116585874, 0.17800126806362837]
 [0.36986660400669347, 0.9618048068583325]
 [0.5461963986546966, 0.6425461373701451]
 [0.2661180892659998, 0.213786475619685]
  ⋮

In [16]:
Polytope_Test = VPolytope(x_test[ch.vertices,:])

VPolytope{Float64, Vector{Float64}}([[0.04516676252777829, 0.001038703028294119, 0.01753097942872539, 0.03057039394926897, 0.06870156478982192, 0.10713956571443861, 0.8630983402270727, 0.9815001890928198, 0.9875465525888553, 0.977669978635453, 0.9701902381663272, 0.9276188833028498, 0.8183391386925531, 0.3732124255692404], [0.9885795089747481, 0.5069685650150854, 0.28689163117415317, 0.19819610321586212, 0.040967439421662366, 0.010555603515481349, 0.009163791471611171, 0.08166064796001371, 0.12845047160605672, 0.5502556827066731, 0.7669756716870548, 0.975094623588664, 0.9888148935893187, 0.9887535001641116]])

In [21]:
LazySets.dim(Polytope_Test)

14

In [18]:
test_points = Vector{Any}([])#Vector{Float64}
a=0
matrix =  clusters[9].x
for i in size(clusters[9].x)[1]
    vector_mid = matrix[i,:]
    push!(test_points, vector_mid)
    print(i)
    a=i
end

40

In [26]:
points = N -> [randn(4) for i in 1:N]
v = points(40)
hull = LazySets.convex_hull(v)
LazySets.dim(VPolytope(hull))
element(Singleton(v[1])) ∈ VPolytope(hull)

glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to re

true

In [38]:
points_sideline[1]

6

In [131]:
x = [1,1,1] # test point
Z = [1 0  # points along columns
     0 1
     0 0]
using JuMP
using CPLEX
m = Model(CPLEX.Optimizer)
set_silent(m)
@variable(m, 0 <= lambda[j=1:size(Z,2)] <= 1)

@constraint(m, inhull[i=1:length(x)], x[i] == sum{Z[i,j]*lambda[j], j = 1:size(Z,2)})
@constraint(m, sum(lambda) == 1)
status = solve(m)

LoadError: LoadError: The curly syntax (sum{},prod{},norm2{}) is no longer supported. Expression: `sum{Z[i, j] * lambda[j], j = 1:size(Z, 2)}`.
in expression starting at c:\Users\32mor\OneDrive\Documents\IPP M2\OR\Projet_RODM_2023\RODM_Project\test_julia_notebook.ipynb:11

In [119]:
∈(x_test[1,:], Polytope_Test)

AssertionError: AssertionError: a vector of length 2 cannot be contained in a polytope of dimension 15

In [105]:
v[ch.vertices]

MethodError: MethodError: no method matching getindex(::MixedMatVRep{Float64, Matrix{Float64}}, ::Vector{Int64})

In [11]:
empty_clusters  = Array{Int64}(undef, 0)
new_clusters = Array{Cluster}(undef, 0)
for i in 1:length(clusters)
    cluster = clusters[i]
    if isempty(cluster.dataIds)
        push!(empty_clusters, i)
    else
        push!(new_clusters, cluster)
    end
end
distances_new = Matrix{Float64}(undef, 0,0)


0×0 Matrix{Float64}